In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from random import seed, shuffle
import os
from datetime import datetime

from pyannote.audio import Pipeline
from pyannote.core import json

from matplotlib import pyplot as plt, lines
import seaborn as sns

from sklearn.cluster import KMeans, SpectralClustering
from sklearn.preprocessing import StandardScaler

from scipy import stats
from scipy.spatial.distance import directed_hausdorff, euclidean, cosine, pdist

from src import (
    downsample as downsamp_audio, 
    embedding_extractor as ee, 
    dimension_reducer as dr, 
    distance_metrics as dm, 
    downsample as ds,
    emb_manipulator as em
)

from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [22]:
##Downsample audio to 16kHz
root_dir= "/Users/rahulbrito/Documents/projects/infantvoice/data/Full_Readings/spont_script/"
audio_files = os.path.join(root_dir,"preprocessed_audios_dur3sec")

#make a folder with today's date for the downsampled audio
down_sample_dir = os.path.join(root_dir,"preprocessed_audios_dur3sec")

#if not os.path.exists(down_sample_dir):
#    os.mkdir(down_sample_dir)

In [ ]:
ds.downsamp_audio(audio_files, down_sample_dir)

processing023.wav


In [28]:
##Diarize audio to find when mother is speaking and visualize it to choose label

diar_dir = os.path.join(root_dir,'diarized')
  
if not os.path.exists(diar_dir):
    os.mkdir(diar_dir)

In [ ]:
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization")

for file in os.listdir(down_sample_dir):
    if file.endswith('wav'):
        diar = pipeline(os.path.join(down_sample_dir,file))
        json.dump_to(diar, os.path.join(diar_dir, os.path.splitext(file)[0]+'.json'))

In [8]:
# Based on output of diarization, manually make speaker map to set which speaker we want
speaker_map = {'021':'SPEAKER_01', '022':'SPEAKER_00', 'BB003':'SPEAKER_00'}

In [9]:
diarized = {os.path.splitext(file)[0]:
            json.load_from(os.path.join(diar_dir,file)).label_timeline(
                speaker_map[os.path.splitext(file)[0]]) 
            for file in os.listdir(diar_dir)}

In [5]:
emb_dir = '/Users/rahulbrito/Documents/projects/infantvoice/data/embeddings'
save_name = datetime.now().strftime('%Y_%m_%d') + '_spont_script_emb.csv'
diarized = None

In [11]:
pyannote_emb = ee.pyannote_extract_directory(down_sample_dir, diarized,emb_dir,save_name,save=True)

Started


ValueError: need at least one array to concatenate

In [10]:
down_sample_dir

'/Users/rahulbrito/Documents/projects/infantvoice/data/Full_Readings/spont_script/preprocessed_audios_dur3sec'

In [27]:
import glob
all_files = glob.glob(down_sample_dir+'/*/*', recursive=True)
all_files = [file for file in all_files if 'script' in file]


In [28]:
len(all_files)

6471

In [29]:
all_files

['/Users/rahulbrito/Documents/projects/infantvoice/data/Full_Readings/spont_script/preprocessed_audios_dur3sec/132/132_M_LPP_script_019.wav',
 '/Users/rahulbrito/Documents/projects/infantvoice/data/Full_Readings/spont_script/preprocessed_audios_dur3sec/132/132_M_LPP_script_025.wav',
 '/Users/rahulbrito/Documents/projects/infantvoice/data/Full_Readings/spont_script/preprocessed_audios_dur3sec/132/132_M_NWS_script_003.wav',
 '/Users/rahulbrito/Documents/projects/infantvoice/data/Full_Readings/spont_script/preprocessed_audios_dur3sec/132/132_M_HT2_script_007.wav',
 '/Users/rahulbrito/Documents/projects/infantvoice/data/Full_Readings/spont_script/preprocessed_audios_dur3sec/132/132_M_HT2_script_013.wav',
 '/Users/rahulbrito/Documents/projects/infantvoice/data/Full_Readings/spont_script/preprocessed_audios_dur3sec/132/132_M_QNA_spon_027.wav',
 '/Users/rahulbrito/Documents/projects/infantvoice/data/Full_Readings/spont_script/preprocessed_audios_dur3sec/132/132_M_QNA_spon_033.wav',
 '/Users/r